In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import skew
from scipy.special import boxcox1p
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [2]:
## Inputting data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

PassengerId = test["PassengerId"]

train.drop(['PassengerId'],axis=1,inplace=True)
test.drop(['PassengerId'],axis=1,inplace=True)

all_data = pd.concat([train.drop('Survived',axis=1),test])
all_data['Title'] = all_data['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]

all_data = all_data.replace({'male': 0, 'female': 1})
all_data.replace('Don','Mr',inplace=True)
all_data.replace(['Mme','Dona'],'Mrs',inplace=True)
all_data.replace(['Ms','Mlle'],'Miss',inplace=True)
all_data.replace(['Major','Col','Capt'],'Crew',inplace=True)
all_data.replace(['Lady','Sir','the Countess','Jonkheer'],'Nob',inplace=True)

all_data['Embarked'].fillna('S')


all_data_copy = all_data.copy()

In [4]:
all_data.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked', 'Title'],
      dtype='object')

In [12]:
## Converting categorical variables to numerical
all_data = pd.concat([all_data,pd.get_dummies(all_data['Embarked'])],axis=1)
all_data = pd.concat([all_data,pd.get_dummies(all_data['Title'])],axis=1)

all_data['Family'] = all_data['SibSp']+all_data['Parch']
all_data['Cabin'] = all_data['Cabin'].isna()
#all_data['Child'] = all_data['Age'].loc(all_data['Age']<16)
all_data['Age'].fillna(all_data.groupby(['Title','Pclass'])['Age'].transform("mean"),inplace=True)
all_data['Fare'].fillna(all_data.groupby(['Pclass','Embarked'])['Fare'].transform("mean"),inplace=True)
all_data['Child'] = all_data['Age']<16
all_data['MothChild'] = all_data['Sex']*all_data['Parch']
all_data['SingAdMale'] = all_data['Mr']*np.heaviside(-all_data_copy['Parch'],1)
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
all_data = all_data[numeric_feats]
all_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,C,Q,S,...,Master,Miss,Mr,Mrs,Nob,Rev,Family,Child,MothChild,SingAdMale
0,3,0,22.0,1,0,7.2500,True,0,0,1,...,0,0,1,0,0,0,1,False,0,1.0
1,1,1,38.0,1,0,71.2833,False,1,0,0,...,0,0,0,1,0,0,1,False,0,0.0
2,3,1,26.0,0,0,7.9250,True,0,0,1,...,0,1,0,0,0,0,0,False,0,0.0
3,1,1,35.0,1,0,53.1000,False,0,0,1,...,0,0,0,1,0,0,1,False,0,0.0
4,3,0,35.0,0,0,8.0500,True,0,0,1,...,0,0,1,0,0,0,0,False,0,1.0


In [15]:
train = pd.concat([all_data.iloc[0:train.shape[0]],train['Survived']],axis=1)
test = all_data.iloc[train.shape[0]:]

In [19]:
y = train['Survived']
X = train.drop('Survived',axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [24]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
print(rf.score(X_train,y_train))
print(rf.score(X_test,y_test))

0.8473053892215568
0.8340807174887892


In [35]:
def plot_feature_importances(model,n_features=10):
    n_features = min(n_features,X_train.shape[1])
    plt.figure(figsize=(8,8))
    plt.barh(range(n_features), np.sort(model.feature_importances_)[::-1][0:n_features], align='center') 
#    plt.yticks(np.arange(n_features), X_train.columns.values[np.argsort(model.feature_importances_)[::-1][0:n_features]])
    print(X_train.columns.values[np.argsort(model.feature_importances_)[::-1][0:n_features]])
#    plt.xlabel('Feature importance')
#    plt.ylabel('Feature')

In [40]:
rf_clf = RandomForestClassifier()
rf_param_grid = {
    'n_estimators': [30,100,300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [ 4, 6, 10],
    'min_samples_split': [ 10,15,20],
    'min_samples_leaf': [3,5],
    'class_weight' : [None,'balanced']
}
rf_grid_search = GridSearchCV(rf_clf, rf_param_grid, cv=3)
rf_grid_search.fit(X_train, y_train)
print(f"Training Accuracy: {rf_grid_search.best_score_ :.2%}")
print("")
print(f"Optimal Parameters: {rf_grid_search.best_params_}")

Training Accuracy: 83.83%

Optimal Parameters: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 300}


In [41]:
train['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [46]:
rf_best = RandomForestClassifier(class_weight=None,criterion='entropy',max_depth=6,min_samples_leaf=5,min_samples_split=10, n_estimators=300)
rf_best.fit(X_train,y_train)
print(rf_best.score(X_train,y_train))
print(rf_best.score(X_test,y_test))

0.844311377245509
0.8295964125560538


In [49]:
predictions = rf_best.predict(test)
submission = pd.DataFrame({"PassengerId":PassengerId, "Survived":predictions})
submission.to_csv("submission_RF.csv", index=False)

## XGBoost

In [70]:
clf = XGBClassifier()

# Fit XGBClassifier
clf.fit(X_train, y_train)

# Predict on training and test sets
training_preds = clf.predict(X_train)
test_preds = clf.predict(X_test)

# Accuracy of training and test sets
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)

print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))

Training Accuracy: 88.77%
Validation accuracy: 82.06%


In [71]:
predictions = xg_best.predict(test)
submission = pd.DataFrame({"PassengerId":PassengerId, "Survived":predictions})
submission.to_csv("submission_XG.csv", index=False)

In [68]:
param_grid = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [6,10],
    'min_child_weight': [1,2,3],
    'subsample': [0.3, 0.5, 0.7],
    'n_estimators': [100,300,1000],
}

grid_clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=3, n_jobs=1)
grid_clf.fit(X_train, y_train)

best_parameters = grid_clf.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train)
test_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)

print('')
print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))

Grid Search found the following optimal parameters: 
learning_rate: 0.2
max_depth: 6
min_child_weight: 2
n_estimators: 300
subsample: 0.3

Training Accuracy: 92.37%
Validation accuracy: 82.51%


In [65]:
xg_best = XGBClassifier(learning_rate=0.2,max_depth=6,min_child_weight=2,n_estimators=300,subsample=0.3)
xg_best.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.2,
       max_delta_step=0, max_depth=6, min_child_weight=2, missing=None,
       n_estimators=300, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=0.3, verbosity=1)

In [69]:
predictions = xg_best.predict(test)
submission = pd.DataFrame({"PassengerId":PassengerId, "Survived":predictions})
submission.to_csv("submission_XG.csv", index=False)

In [72]:
X_train.shape

(668, 22)